In [2]:
from methods import *
import nltk
nltk.download('punkt_tab')
from nltk import word_tokenize
import fasttext
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import random
import re

[nltk_data] Downloading package tagsets_json to
[nltk_data]     /Users/tscreven/nltk_data...
[nltk_data]   Package tagsets_json is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tscreven/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
BASE_NAMES = basenames()
VOTE_THRESHOLD = 1
SPLIT = 0.8
WORD_RE = re.compile(r'.*\w')

In [ ]:
def tokenize(all_text):
    all_tokens = []
    for rev_text in all_text:
        tokens = word_tokenize(rev_text)
        token_string = ' '.join(token for token in tokens if WORD_RE.match(token))
        all_tokens.append(token_string)
    return all_tokens

def get_features(filename):
    text, labels = get_data(filename, VOTE_THRESHOLD)
    train_text, train_labels, test_text, test_labels = split_data(text, labels, SPLIT)
    train_tokens = tokenize(train_text)
    test_tokens = tokenize(test_text)

    return train_tokens, train_labels, test_tokens, test_labels

In [5]:
def generate_file(base_name, folder):
    train_file = open(f'{folder}/{base_name}_train.ft', 'w')
    train_tokens, train_labels, test_tokens, test_labels = get_features(get_decimated_name(base_name))
    for i in range(len(train_tokens)):
        train_file.write(f'__label__{train_labels[i]} {train_tokens[i]}\n')

    train_file.close()
    return test_tokens, test_labels

In [ ]:
def run(base, folder):
    test_tokens, test_labels = generate_file(base, folder)
    model = fasttext.train_supervised(f'{folder}/{base}_train.ft', epoch=12, dim=125, verbose=0)
    output = model.predict(test_tokens)
    predictions = output[0]
    scocres = output[1]
    processed_predicitons = [int(x[0][-1]) for x in predictions]
    processed_scores = [s[0].item() if p==1 else 1-s[0].item() for s,p in zip(scores, processed_predicitons)]
    results = [(l, s) for l,s in zip(test_labels, processed_scores)]
    results = sorted(results, key=lambda x: x[1], reverse=True)
    top_size = int(len(results) * 0.05)
    top_percent = results[:top_size]
    true_positives = sum(1 for label, _ in top_percent if label == 1)
    precision_top_5_percent = true_positives / top_size
    return precision_top_5_percent


0.7563025210084033

In [11]:
for base in BASE_NAMES:
    print(run(base, 'TextOnly'))

1.0
0.7857142857142857


KeyboardInterrupt: 